GRU model


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#naitame data
train_data = pd.read_csv("/content/drive/MyDrive/toxic_eng/train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/toxic_eng/test.csv")

# otestujeme ci neobsahuje
train_data['label'] = train_data.get('toxic')
test_data['label'] = test_data.get('toxic')

# tokenizacia
max_words = 20000
max_length = 128
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['comment_text'])

train_sequences = tokenizer.texts_to_sequences(train_data['comment_text'])
test_sequences = tokenizer.texts_to_sequences(test_data['comment_text'])

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

# konvertujeme na numpy
train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

#  GRU model
model = Sequential([
    Embedding(max_words, 128, input_length=max_length),
    GRU(128, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")  # Binary classification
])

model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

#trenovanie
model.fit(train_padded, train_labels, validation_data=(test_padded, test_labels), epochs=20, batch_size=16)

test_loss, test_acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {test_acc:.4f}")





In [ ]:
model.save("/content/drive/MyDrive/saved_gru_model.h5")


In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/toxic_eng/train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/toxic_eng/test.csv")


train_data['label'] = train_data.get('toxic')
test_data['label'] = test_data.get('toxic')


In [ ]:
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['comment_text'])

In [ ]:
tokenizer


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

model_path = "/content/drive/MyDrive/saved_gru_model.h5"
model = tf.keras.models.load_model(model_path)

model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

dataset_path = "/content/drive/MyDrive/combined_dataset.csv"
toxic_data = pd.read_csv(dataset_path)

toxic_data.columns = toxic_data.columns.str.strip()
print("**Opravené názvy stĺpcov:**", toxic_data.columns)

print(" **Prvých 5 riadkov datasetu:**")
print(toxic_data.head())

print(" **Počet NaN hodnôt v label:**", toxic_data["label"].isna().sum())

toxic_data = toxic_data.dropna(subset=["label"])

toxic_data["label"] = toxic_data["label"].astype(int)

print(f" **Počet komentárov po odstránení NaN:** {len(toxic_data)}")

print(" **Distribúcia hodnôt label:**")
print(toxic_data["label"].value_counts())

if toxic_data["label"].nunique() == 1:
    print(" **Varovanie:** Všetky vzorky majú rovnaký label, stratifikácia sa vypína.")
    stratify_param = None
else:
    stratify_param = toxic_data["label"]

train_texts, test_texts, train_labels, test_labels = train_test_split(
    toxic_data["text"], toxic_data["label"], test_size=0.2, random_state=42, stratify=stratify_param
)

print(f" **Tréningové vzorky:** {len(train_texts)}")
print(f" **Testovacie vzorky:** {len(test_texts)}")


In [ ]:
max_vocab_size = 20000  # rovnake ako pri pôvodnom trénovaní
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<UNK>")

tokenizer.fit_on_texts(train_texts)

# text na cisla
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# nastavime max dlzku sekvencií (padding)
max_length = 128
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

# prekonvertujeme si labely na numpy array
train_labels = np.array(train_labels).astype("float32")
test_labels = np.array(test_labels).astype("float32")

print("\n **Príklad tokenizovaných sekvencií (prvých 5):**")
for i in range(5):
    print(f" {train_texts.iloc[i]} → {train_sequences[i]}")


In [ ]:
#trening modelu
history = model.fit(
    train_padded, train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(test_padded, test_labels)
)

# vyhodnotenie na testovacich datach
test_loss, test_accuracy = model.evaluate(test_padded, test_labels)
print(f"\n**Testovacia presnosť:** {test_accuracy:.4f}, **Testovacia strata:** {test_loss:.4f}")

# Predikcie na testovacom sete
predictions = model.predict(test_padded)
predictions = (predictions > 0.5).astype(int)  # Konvertujeme pravdepodobnosti na 0 alebo 1

# vyhodnotenie s confusion matrix
print("\n **Vyhodnotenie:**")
print(classification_report(test_labels, predictions))

print("\n **Matica chýb:**")
print(confusion_matrix(test_labels, predictions))


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
MAX_LEN = 128

train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len=MAX_LEN)
test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_len=MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


LSTM MODEL


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import pickle
from torch.utils.data import DataLoader, TensorDataset, random_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
  # pouzijemem tu istu architekturu ako to bolo pri anglickych datach
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, num_layers, dropout):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.dropout(x[:, -1, :])
        x = self.fc(x)
        return x


vocab_size = tokenizer.vocab_size
embed_dim = 128
hidden_dim = 256
output_dim = 2
num_layers = 2
dropout = 0.3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lstm_model = LSTMClassifier(vocab_size, embed_dim, hidden_dim, output_dim, num_layers, dropout).to(device)

model_path = "/content/drive/MyDrive/lstm_model_final.pth"
try:
    lstm_model.load_state_dict(torch.load(model_path, map_location=device))
    lstm_model.eval()
    print("LSTM model úspešne načítaný.")
except FileNotFoundError:
    print("Model nenájdený! Nahraj `lstm_model.pth` do Drive.")


In [ ]:
import torch
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zariadenie: {device}")

#  tokenizer pre BERT (rovnaký ako pri trénovaní)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
print(" BERT tokenizer úspešne načítaný.")

test_data_path = "/content/drive/MyDrive/combined_dataset.csv"
toxic_data = pd.read_csv(test_data_path)

toxic_data.columns = toxic_data.columns.str.strip()
toxic_data = toxic_data.dropna(subset=["text", "label"])
toxic_data["label"] = toxic_data["label"].astype(int)

print(f" **Celkovy pocet testovacich vzoriek:** {len(toxic_data)}")
print(" **distribucia hodnot label:**")
print(toxic_data["label"].value_counts())

max_length = 128
encoded_texts = tokenizer(
    list(toxic_data["text"]),
    padding="max_length",
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

input_ids = encoded_texts["input_ids"].to(device)
attention_masks = encoded_texts["attention_mask"].to(device)
test_labels = torch.tensor(toxic_data["label"].values, dtype=torch.long).to(device)

batch_size = 32
test_dataset = TensorDataset(input_ids, attention_masks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model_path = "/content/drive/MyDrive/lstm_model_final.pth"
try:
    lstm_model.load_state_dict(torch.load(model_path, map_location=device))
    lstm_model.eval()
    print(" LSTM model úspešne načítaný.")
except FileNotFoundError:
    print(" Model nenájdený! Nahraj `lstm_model_final.pth` do Drive.")
    exit()

lstm_model.eval()
all_preds, all_labels = [], []
# Nepotrebujeme gradienty pri testovaní
with torch.no_grad():
    for input_ids, attention_masks, labels in tqdm(test_loader, desc=" Testovanie modelu"):
        outputs = lstm_model(input_ids)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"\n **Testovacia presnosť:** {accuracy:.4f}")

# klasifikacia a matica
print("\n **Klasifikacia:**")
print(classification_report(all_labels, all_preds))

print("\n **Matica chýb:**")
print(confusion_matrix(all_labels, all_preds))

print("\n **DOKONcene!** Model bol úspešne otestovaný.")


BERT MODEL


In [ ]:
#DOTRENOVAVANIEBERTU NA SLOVENSKYCH DATACH
#ukoncilo sa mi to na 9 epoche
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

#  Zariadenie
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používam: {device}")

#  Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

data_path = "/content/drive/MyDrive/combined_dataset.csv"
data = pd.read_csv(data_path)
data.columns = data.columns.str.strip()
data = data.dropna(subset=["text", "label"])
data["label"] = data["label"].astype(int)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"], data["label"], test_size=0.2, random_state=42, stratify=data["label"]
)

#  Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, idx):
        encoded = self.tokenizer.encode_plus(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded['input_ids'].squeeze(0),
            'attention_mask': encoded['attention_mask'].squeeze(0),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.texts)

#  Dataloaders
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

#  definicia modelu
class BERTClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        dropped = self.dropout(pooled)
        return self.classifier(dropped)

model = BERTClassifier().to(device)
checkpoint_path = "/content/drive/MyDrive/bert_model.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
print("Načítaný predtrénovaný BERT model.")

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# dotrenovanie pouzitim early stopping
model.train()
num_epochs = 20
patience = 3
best_acc = 0.0
epochs_no_improve = 0

for epoch in range(num_epochs):
    total_loss, correct, total = 0, 0, 0
    model.train()

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), "/content/drive/MyDrive/toxic_sk/best_bert_model_sk.pth")
        print(" → Zlepšenie! Model uložený.")
    else:
        epochs_no_improve += 1
        print(f" → Žiadne zlepšenie ({epochs_no_improve}/{patience})")

    if epochs_no_improve >= patience:
        print("\n  Tréning ukončený kvôli nedostatku zlepšenia (early stopping).")
        break

# Vyhodnotenie
model.load_state_dict(torch.load("/content/drive/MyDrive/toxic_sk/best_bert_model_sk.pth", map_location=device))
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(f"\n Presnosť na SK testovacích dátach: {accuracy_score(all_labels, all_preds):.4f}")
print("\nKlasifikačná správa:")
print(classification_report(all_labels, all_preds))
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

final_path = "/content/drive/MyDrive/toxic_sk/fine_tuned_bert_model_sk.pth"
torch.save(model.state_dict(), final_path)
print(f"\nFinálny model uložený do: {final_path}")


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používam: {device}")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

data_path = "/content/drive/MyDrive/combined_dataset.csv"
data = pd.read_csv(data_path)
data.columns = data.columns.str.strip()
data = data.dropna(subset=["text", "label"])
data["label"] = data["label"].astype(int)

# rozdelenie dat
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"], data["label"], test_size=0.2, random_state=42, stratify=data["label"]
)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, idx):
        encoded = self.tokenizer.encode_plus(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded['input_ids'].squeeze(0),
            'attention_mask': encoded['attention_mask'].squeeze(0),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.texts)

test_dataset = TextDataset(test_texts, test_labels, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16)

#  BERT model
class BERTClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        dropped = self.dropout(pooled)
        return self.classifier(dropped)

model = BERTClassifier().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/toxic_sk/best_bert_model_sk.pth", map_location=device))
model.eval()

# Vyhodnotenie
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testovanie"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# vyhodnotenie
print(f"\nPresnosť na SK testovacích dátach: {accuracy_score(all_labels, all_preds):.4f}")
print("\nKlasifikačná správa:")
print(classification_report(all_labels, all_preds))
print("\nMaticová konfúzia:")
print(confusion_matrix(all_labels, all_preds))

final_path = "/content/drive/MyDrive/toxic_sk/226181.pth"
torch.save(model.state_dict(), final_path)
print(f"\nModel bol uložený do: {final_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Používam: cuda


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Testovanie: 100%|██████████| 111/111 [00:11<00:00,  9.28it/s]



Presnosť na SK testovacích dátach: 0.8967

Klasifikačná správa:
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       883
           1       0.94      0.84      0.89       879

    accuracy                           0.90      1762
   macro avg       0.90      0.90      0.90      1762
weighted avg       0.90      0.90      0.90      1762


Maticová konfúzia:
[[839  44]
 [138 741]]

Model bol uložený do: /content/drive/MyDrive/toxic_sk/226181.pth


In [ ]:
#mt5 dotrenovanie

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


from google.colab import drive
drive.mount('/content/drive')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používam: {device}")

# tokenizer
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")

# nacitanie dat
data_path = "/content/drive/MyDrive/combined_dataset.csv"
data = pd.read_csv(data_path)
data.columns = data.columns.str.strip()
data = data.dropna(subset=["text", "label"])
data["label"] = data["label"].astype(str)

# rozdelenie dat
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"], data["label"], test_size=0.2, random_state=42, stratify=data["label"]
)

# Dataset
class MT5TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, idx):
        input_encoding = self.tokenizer(
            self.texts[idx],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            self.labels[idx],
            max_length=2,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_encoding["attention_mask"].squeeze(),
            "labels": target_encoding["input_ids"].squeeze()
        }

    def __len__(self):
        return len(self.texts)

train_dataset = MT5TextDataset(train_texts, train_labels, tokenizer)
test_dataset = MT5TextDataset(test_texts, test_labels, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small").to(device)
checkpoint_path = "/content/drive/MyDrive/mt5_toxicity_full_epoch10.py"
try:
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print("Načítaný predtrénovaný mT5 model.")
except:
    print("Predtrénovaný model nenačítaný, bude použitý nový.")

# trening
optimizer = optim.AdamW(model.parameters(), lr=3e-5)
num_epochs = 20
patience = 3
best_acc = 0.0
epochs_no_improve = 0

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")

    # validacia
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=2)
            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            all_preds.extend(decoded_preds)
            all_labels.extend(decoded_labels)

    acc = accuracy_score(all_labels, all_preds)
    print(f"Test presnosť: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), "/content/drive/MyDrive/toxic_sk/best_mt5_model_sk.pth")
        print(" → Zlepšenie! Model uložený.")
    else:
        epochs_no_improve += 1
        print(f" → Žiadne zlepšenie ({epochs_no_improve}/{patience})")

    if epochs_no_improve >= patience:
        print("\n  Tréning ukončený kvôli nedostatku zlepšenia (early stopping).")
        break
    model.train()

# Záverečné vyhodnotenie
model.load_state_dict(torch.load("/content/drive/MyDrive/toxic_sk/best_mt5_model_sk.pth", map_location=device))
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=2)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        all_preds.extend(decoded_preds)
        all_labels.extend(decoded_labels)

print(f"\n Presnosť na SK testovacích dátach: {accuracy_score(all_labels, all_preds):.4f}")
print("\nKlasifikačná správa:")
print(classification_report(all_labels, all_preds))
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

# Finálne uloženie
final_path = "/content/drive/MyDrive/toxic_sk/fine_tuned_mt5_model_sk.pth"
torch.save(model.state_dict(), final_path)
print(f"\nFinálny model uložený do: {final_path}")


Mounted at /content/drive
Používam: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

Epoch 18 - Loss: 307.2924
Test presnosť: 0.7446
 → Zlepšenie! Model uložený.


Epoch 19/20: 100%|██████████| 881/881 [03:08<00:00,  4.67it/s]


Epoch 19 - Loss: 288.7359
Test presnosť: 0.7571
 → Zlepšenie! Model uložený.


Epoch 20/20: 100%|██████████| 881/881 [03:07<00:00,  4.69it/s]


Epoch 20 - Loss: 278.3262
Test presnosť: 0.7764
 → Zlepšenie! Model uložený.

 Presnosť na SK testovacích dátach: 0.7764

Klasifikačná správa:
              precision    recall  f1-score   support

         0.0       0.76      0.82      0.79       883
         1.0       0.80      0.73      0.77       879

    accuracy                           0.78      1762
   macro avg       0.78      0.78      0.78      1762
weighted avg       0.78      0.78      0.78      1762


Confusion Matrix:
[[723 160]
 [234 645]]

Finálny model uložený do: /content/drive/MyDrive/toxic_sk/fine_tuned_mt5_model_sk.pth


byT5 MODEL

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import ByT5Tokenizer, ByT5Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = ByT5Tokenizer.from_pretrained("google/byt5-small")

data = pd.read_csv("/content/drive/MyDrive/combined_dataset.csv")
data.columns = data.columns.str.strip()
data = data.dropna(subset=["text", "label"])
data["label"] = data["label"].astype(int)

dtrain_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"].tolist(), data["label"].tolist(),
    test_size=0.2, random_state=42, stratify=data["label"]
)

# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Dataloaders
train_dataset = TextDataset(dtrain_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# ByT5 klasifikačný model
class ByT5ForClassification(nn.Module):
    def __init__(self, num_labels=2):
        super(ByT5ForClassification, self).__init__()
        self.byt5 = ByT5Model.from_pretrained("google/byt5-small")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.byt5.config.d_model, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.byt5(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        pooled_output = hidden_states.mean(dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits
# nacitnaie predtrenovaneho modelu
model = ByT5ForClassification(num_labels=2).to(device)
eng_model_path = "/content/drive/MyDrive/byt5_toxicity_best.pt"
checkpoint = torch.load(eng_model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
print("Anglický model ByT5 načítaný.")

# trening
optimizer = optim.Adam(model.parameters(), lr=3e-5)
criterion = nn.CrossEntropyLoss()
num_epochs = 20
patience = 3
best_acc = 0.0
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f" Epoch {epoch+1} - Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        epochs_no_improve = 0
        torch.save({'model_state_dict': model.state_dict()},
                   "/content/drive/MyDrive/toxic_sk/best_byt5_finetuned_sk.pth")
        print(" → Zlepšenie! Model uložený.")
    else:
        epochs_no_improve += 1
        print(f" → Žiadne zlepšenie ({epochs_no_improve}/{patience})")

    if epochs_no_improve >= patience:
        print("Tréning ukončený kvôli early stopping.")
        break

# Vyhodnotenie
checkpoint = torch.load("/content/drive/MyDrive/toxic_sk/best_byt5_finetuned_sk.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
print(f"\n Presnosť (Accuracy) na SK testovacích dátach: {acc:.4f}")
print("\n Výsledky klasifikácie:")
print(classification_report(all_labels, all_preds, digits=4))
print(" Maticová konfúzia:")
print(confusion_matrix(all_labels, all_preds))
